In [1]:
import torch
import pandas as pd
import warnings
import os
from pandas.errors import SettingWithCopyWarning


# 忽略特定類型的警告
warnings.filterwarnings("ignore", category=FutureWarning, message="ChainedAssignmentError")
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

# Constants

In [2]:
GPUS = "0,1,5,3,2,4"
REORDER = False
RAW_DATA_DIR = '../../cleaned_dataset/'
FEAT_DATA_DIR = './tmp/feat/'
DREAM_MODEL_DIR = './tmp/dream/'

if not os.path.exists(FEAT_DATA_DIR):
    os.makedirs(FEAT_DATA_DIR)
if not os.path.exists(DREAM_MODEL_DIR):
    os.makedirs(DREAM_MODEL_DIR)

DREAM_CONFIG = {'basket_pool_type': 'max', # 'avg'
                'rnn_layers': 2,  # TaFeng:2, Dunnhumby:2
                'rnn_type': 'RNN_RELU',  #'RNN_TANH',#'GRU',#'LSTM',# 'RNN_RELU', TaFeng: RNN_RELU, Dunnhumby: ,Instacart:
                'dropout': 0.5,
                'num_product': 3003+1, # TaFeng:12085+1, Dunnhumby:3003+1
                'none_idx':  3003, # TaFeng:12085, Dunnhumby:3003
                'embedding_dim': 128, # TaFeng: 128 
                'cuda': torch.cuda.is_available(), # True,
                'clip': 200,  # 0.25
                'epochs': 40,   # 40
                'batch_size': 32, # TaFeng: 32, Dunnhumby: 32
                'learning_rate': 0.001, # TaFeng: 0.001, Dunnhumby: 0.001
                'log_interval': 2, # num of batchs between two logging
                'data_name': "Dunnhumby", # 資料集名稱  TaFeng, Dunnhumby
                'train_times': 1, # 訓練次數
                # 'checkpoint_dir': DREAM_MODEL_DIR + '/{data_name}/' + '\dream-{epoch:02d}-{loss:.4f}.model',
                }

# Config

In [3]:
"""
 configure DREAM
 
 @TODO: REFACTOR config class
"""
       
class Config(object):
    def __init__(self, config):
        self.cuda = config['cuda']
        self.clip = config['clip']
        self.epochs = config['epochs']
        self.batch_size = config['batch_size']
        self.learning_rate = config['learning_rate'] # Initial Learning Rate
        self.log_interval = config['log_interval']
        
        self.none_idx = config['none_idx']
        # self.pn_pair_num = config['pn_pair_num']
        self.basket_pool_type = config['basket_pool_type']
        self.rnn_type = config['rnn_type']
        self.rnn_layer_num = config['rnn_layers']
        self.dropout = config['dropout']
        self.num_product = config['num_product'] # 商品数目，用于定义Embedding Layer
        self.embedding_dim = config['embedding_dim'] # 商品表述维数， 用于定义Embedding Layer
        self.data_name = config['data_name']
        self.train_times = config['train_times']
        # self.checkpoint_dir = config['checkpoint_dir']

        self.alias = '{}{}_{}_{}_{}_clip{}'.format(self.rnn_layer_num,
                                                   self.rnn_type,
                                                   self.embedding_dim,
                                                   self.learning_rate,
                                                   self.batch_size,
                                                   self.clip)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Utils

In [5]:
# -*- coding: utf-8 -*-
"""
    Useful Functions
"""
import torch
from torch.autograd import Variable

import numpy as np

# import constants
# from config import Config
# dr_config = Config(constants.DREAM_CONFIG)
dr_config = Config(DREAM_CONFIG)

################### PADDING & BATCH
def pad(tensor, length):
    '''
        pad 1st dim
        remain 0th 2nd dim
    '''
    #pdb.set_trace()
    return torch.cat([tensor, tensor.new(tensor.size(0), length - tensor.size(1), *tensor.size()[2:]).zero_()], 1)

def sort_batch_of_lists(batch_of_lists, lens, uids, rbks = None, ihis = None):
    '''
        sort batch of lists according to len(list)
        descending
    '''
    sorted_idx = [i[0] for i in sorted(enumerate(lens), key = lambda x : x[1], reverse = True)]
    uids = [uids[i] for i in sorted_idx]
    lens = [lens[i] for i in sorted_idx]
    batch_of_lists = [batch_of_lists[i] for i in sorted_idx]
    if rbks is not None and ihis is not None:
        rbks = [rbks[i] for i in sorted_idx]
        ihis = [ihis[i] for i in sorted_idx]
        return batch_of_lists, lens, uids, rbks, ihis
    else:
        return batch_of_lists, lens, uids

def pad_batch_of_lists(batch_of_lists, max_len):
    '''
        pad batch of lists
    '''
    padded = [l + [[dr_config.none_idx]] * (max_len - len(l)) for l in batch_of_lists]
    return padded
    
def batchify(data, batch_size, is_reordered = False):
    '''
        turn dataset into iterable batches
    '''
    if is_reordered is True:
        num_batch = len(data) // batch_size
        for i in range(num_batch):
            baskets, lens, uids, rbks, ihis = data[i * batch_size : (i + 1) * batch_size] # load
            baskets, lens, uids, rbks, ihis = sort_batch_of_lists(baskets, lens, uids, rbks, ihis) # sort, max_len = lens[0]
            padded_baskets = pad_batch_of_lists(baskets, lens[0]) # pad
            padded_rbks = pad_batch_of_lists(rbks, lens[0])
            padded_ihis = pad_batch_of_lists(ihis, lens[0])
            yield padded_baskets, lens, uids, padded_rbks, padded_ihis
        
        if len(data) % batch_size != 0:
            residual = [i for i in range(num_batch * batch_size, len(data))] + list(np.random.choice(len(data), batch_size - len(data) % batch_size))    
            print(len(residual))
            baskets, lens, uids, rbks, ihis = map(list, zip(*[data[idx] for idx in residual]))
            baskets, lens, uids, rbks, ihis = sort_batch_of_lists(baskets, lens, uids, rbks, ihis)
            padded_baskets = pad_batch_of_lists(baskets, lens[0])
            padded_rbks = pad_batch_of_lists(rbks, lens[0])
            padded_ihis = pad_batch_of_lists(ihis, lens[0])
            yield padded_baskets, lens, uids, padded_rbks, padded_ihis

    else: 
        num_batch = len(data) // batch_size
        for i in range(num_batch):
            baskets, lens, uids = data[i * batch_size : (i + 1) * batch_size] # load
            baskets, lens, uids = sort_batch_of_lists(baskets, lens, uids) # sort, max_len = lens[0]
            padded_baskets = pad_batch_of_lists(baskets, lens[0]) # pad
            yield padded_baskets, lens, uids
        
        if len(data) % batch_size != 0:
            residual = [i for i in range(num_batch * batch_size, len(data))] + list(np.random.choice(len(data), batch_size - len(data) % batch_size))    
            print(len(residual))
            baskets, lens, uids = map(list, zip(*[data[idx] for idx in residual]))
            baskets, lens, uids = sort_batch_of_lists(baskets, lens, uids)
            padded_baskets = pad_batch_of_lists(baskets, lens[0])
            yield padded_baskets, lens, uids

####################### Model Related

def pool_max(tensor, dim):
    return torch.max(tensor, dim)[0]

def pool_avg(tensor, dim):
    return torch.mean(tensor, dim)

def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
###################### Summary
def get_grad_norm(model):
    grads = []
    for p in model.parameters():
        if p.grad is not None:
            grads.append(p.grad.data.view(-1, 1))
    assert len(grads) > 0
    return torch.norm(torch.cat(grads))


def get_loss(loss):
    """Convert variable into float"""
    loss = loss.data
    if isinstance(loss, torch.cuda.FloatTensor):
        loss = loss.cpu()
    assert len(loss.numpy()) == 1, 'length of loss is large than 1'
    return float(loss.numpy()[0])


def get_ratio_update(delta, weight):
    delta = torch.cat([d.data.view(-1, 1) for d in delta])
    weight = torch.cat([w.data.view(-1, 1) for w in weight])
    return torch.norm(delta) / torch.norm(weight)


def get_weight_update(previous_weight, weight):
    """
    args:
        previous_weight: list
    """
    deltas = []
    for i, w in enumerate(weight):
        deltas.append(w - previous_weight[i])
    return deltas


# Data

In [6]:
# -*- coding: utf-8 -*-
"""
    Prepare Input for DREAM
"""
import os
import pickle
import numpy as np
class BasketConstructor(object):
    '''
        Group products into baskets(type: list)
    '''
    def __init__(self, raw_data_dir, cache_dir):
        self.raw_data_dir = raw_data_dir
        self.cache_dir = cache_dir
    
    def get_users_products(self, ub_basket, df):
        '''
            get users' all purchased products
        '''
        flatten_list = lambda y:[x for a in y for x in flatten_list(a)] if type(y) is list else [y]
        users_products = ub_basket[['user_id','basket']]
        users_products.columns = ['CUSTOMER_ID','NEW_ITEM_ID']
        for i, ub in enumerate(ub_basket['basket']):
            print("flatten", len(set(flatten_list(ub))))
            print("basket", users_products.loc[ i, "NEW_ITEM_ID"])
            users_products.loc[ i, "NEW_ITEM_ID"] =  list( set(flatten_list(ub)) )
            # users_products['NEW_ITEM_ID'][i] =  list( set(flatten_list(ub)) )
        return users_products
    
    def get_baskets(self, data_name, reconstruct = False, reordered = False, none_idx = 49689):
        '''
            get users' baskets

        '''
        if not os.path.exists(f'{self.cache_dir}/{data_name}/'):
             os.mkdir(f'{self.cache_dir}/{data_name}/')
        filepath = f'{self.cache_dir}/{data_name}/basket_{data_name}.pkl'
        print("filepath=",filepath)

        if (not reconstruct) and os.path.exists(filepath):
            with open(filepath, 'rb') as f:
                up_basket = pickle.load(f)
        else:
            df = pd.read_csv(self.raw_data_dir + f'{data_name}_clean.csv') 
            up = df.sort_values(['CUSTOMER_ID', 'CART_ID', 'NEW_ITEM_ID'], ascending = True)
            up_basket = up.groupby(['CUSTOMER_ID', 'CART_ID'])['NEW_ITEM_ID'].apply(list).reset_index()
            up_basket = up_basket.sort_values(['CUSTOMER_ID', 'CART_ID'], ascending = True).groupby(['CUSTOMER_ID'])['NEW_ITEM_ID'].apply(list).reset_index()
            up_basket.columns = ['user_id', 'basket']
            with open(filepath, 'wb') as f:
                pickle.dump(up_basket, f, pickle.HIGHEST_PROTOCOL)

        return up_basket


class Dataset(object):
    '''
        Dataset prepare from user-basket
    '''
    def __init__(self, up_basket, up_r_basket = None, up_his = None):
        if (up_r_basket is not None) and (up_his is not None):
            self.is_reordered_included = True
        else:
            self.is_reordered_included = False

        up_basket['num_baskets'] = up_basket.basket.apply(len)
        self.user_id = list(up_basket.user_id)
        self.num_baskets = [int(n) for n in list(up_basket.num_baskets)]    
        self.basket = [[[int(p) for p in b]for b in u] for u in list(up_basket.basket)]

        if self.is_reordered_included is True:
            up_basket = pd.merge(up_basket, up_r_basket, on = ['user_id'], how = 'left')
            up_basket = pd.merge(up_basket, up_his, on = ['user_id'], how = 'left')
            self.reorder_basket = [[[int(p) for p in b]for b in u] for u in list(up_basket.reorder_basket)]
            self.history_item = [[[int(p) for p in b]for b in u] for u in list(up_basket.history_items)]

    def __getitem__(self, index):
        '''
            return baskets & num_baskets
        '''
        if self.is_reordered_included is True:
            return self.basket[index], self.num_baskets[index], self.user_id[index], self.reorder_basket[index], self.history_item[index]
        else:
            return self.basket[index], self.num_baskets[index], self.user_id[index]

    
    def __len__(self):
        return len(self.user_id)    

# Dream

In [7]:
# -*- coding: utf-8 -*-
"""
    DREAM: Dynamic Recurrent Network for Next Basket Prediction
    Based on PyTorch
    
    @TODO: 
        - Efficiency Variabel Usage
        - Other initilizations for item/product embedding, for example LDA
"""
import random
import numpy as np

import torch
from torch.autograd import Variable
# from utils import pool_max, pool_avg


class DreamModel(torch.nn.Module):
    """       Input Data: b_1, ... b_i ..., b_t
                   b_i stands for user u's ith basket
                   b_i = [p_1,..p_j...,p_n]
                   p_j stands for the  jth product in user u's ith basket
    """
    def __init__(self, config):
        super(DreamModel, self).__init__()
        # Model configuration
        self.config = config
        # Layer definitons
        self.encode = torch.nn.Embedding(config.num_product, 
                                         config.embedding_dim,
                                         padding_idx = config.none_idx) # Item embedding layer, 商品编码
        print("self.encode=",self.encode )
        self.pool = {'avg':pool_avg, 'max':pool_max}[config.basket_pool_type] # Pooling of basket
        # RNN type specify
        if config.rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(torch.nn, config.rnn_type)(config.embedding_dim, 
                                                          config.embedding_dim, 
                                                          config.rnn_layer_num, 
                                                          batch_first=True, 
                                                          dropout=config.dropout)
        else:
            nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[config.rnn_type]
            self.rnn = torch.nn.RNN(config.embedding_dim, 
                                    config.embedding_dim, 
                                    config.rnn_layer_num, 
                                    nonlinearity=nonlinearity, 
                                    batch_first=True, 
                                    dropout=config.dropout)
    
    def forward(self, x, lengths, hidden):
        # Basket Encoding 
        # print("x=\n",x)
        ub_seqs = [] # users' basket sequence
        for user in x: # x shape (batch of user, time_step, indice of product) nested lists
            embed_baskets = []
            for basket in user:
                basket = torch.LongTensor(basket).resize_(1, len(basket))
                basket = basket.cuda() if self.config.cuda else basket # use cuda for acceleration
                basket = self.encode(torch.autograd.Variable(basket)) # shape: 1, len(basket), embedding_dim
                embed_baskets.append(self.pool(basket, dim=1))
            # concat current user's all baskets and append it to users' basket sequence
            ub_seqs.append(torch.cat(embed_baskets, 0).unsqueeze(0))  # shape: 1, num_basket, embedding_dim
        # Input for rnn 
        ub_seqs = torch.cat(ub_seqs, 0).cuda() if self.config.cuda else torch.cat(ub_seqs, 0) # shape: batch_size, max_len, embedding_dim
        packed_ub_seqs = torch.nn.utils.rnn.pack_padded_sequence(ub_seqs, lengths, batch_first=True) # packed sequence as required by pytorch
        
        # RNN
        output, h_u = self.rnn(packed_ub_seqs, hidden)
        dynamic_user, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True) # shape: batch_size, max_len, embedding_dim
        return dynamic_user, h_u
        
    def init_weight(self):
        # Init item embedding
        initrange = 0.1
        self.encode.weight.data.uniform_(-initrange, initrange)
    
    def init_hidden(self, batch_size):
        # Init hidden states for rnn
        weight = next(self.parameters()).data
        if self.config.rnn_type == 'LSTM':
            return (Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_()),
                    Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_()))
        else:
            return Variable(weight.new(self.config.rnn_layer_num, batch_size, self.config.embedding_dim).zero_())   

# Eval

In [8]:
"""
    Evaluation of DREAM
"""
import torch
from math import ceil
from time import time

# from data import Dataset, BasketConstructor
# from utils import repackage_hidden, batchify

import pdb

def eval_pred(dr_model, ub):
    '''
        evaluate dream model for predicting next basket on all training users
        in batches
    '''
    item_embedding = dr_model.encode.weight
    dr_model.eval()
    dr_hidden = dr_model.init_hidden(dr_model.config.batch_size)
    start_time = time()
    id_u, score_u = [], [] # user's id, user's score
    num_batchs = ceil(len(ub) / dr_model.config.batch_size)
    for i,x in enumerate(batchify(ub, dr_model.config.batch_size)):
        print(i)
        baskets, lens, uids = x
        _, dynamic_user, _ = dr_model(baskets, lens, dr_hidden)# shape: batch_size, max_len, embedding_size
        dr_hidden = repackage_hidden(dr_hidden)
        for i,l,du in zip(uids, lens, dynamic_user):
            du_latest = du[l - 1].unsqueeze(0) # shape: 1, embedding_size
            score_up = torch.mm(du_latest, item_embedding.t()) # shape: 1, num_item
            score_u.append(score_up.cpu().data.numpy())
            id_u.append(i)
    elapsed = time() - start_time 
    print('[Predicting] Elapsed: {02.2f}'.format(elapsed))
    return score_u, id_u

def eval_batch(dr_model, ub, batch_size , is_reordered = False):
    '''
        Using dr_model to predict (u,p) score in batch
        Parameters:
        - ub: users' baskets
        - up: users' history purchases
        - batch_size
    '''
    # turn on evaluation mode
    dr_model.eval()
    is_cuda =  dr_model.config.cuda
    item_embedding = dr_model.encode.weight
    dr_hidden = dr_model.init_hidden(batch_size)

    id_u, item_u, score_u, dynam_u = [], [], [], []
    start_time = time()
    num_batchs = ceil(len(ub) / batch_size)
    for i, x in enumerate(batchify(ub, batch_size, is_reordered)):
        if is_reordered is True:
            baskets, lens, uids, r_baskets, h_baskets = x
        else:
            baskets, lens, uids = x
        dynamic_user, _ = dr_model(baskets, lens, dr_hidden)
        for uid, l, du in zip(uids, lens, dynamic_user):
            du_latest =  du[l - 1].unsqueeze(0)
            ### 計算所有項目分數
            all_item = torch.tensor([i for i in range( dr_model.config.none_idx )], dtype=torch.long).to(device)
            # all_item = torch.cuda.LongTensor([i for i in range( dr_model.config.none_idx )])
            score_up = torch.mm(du_latest, item_embedding.t()).cpu().data.numpy()[0]
            id_u.append(uid), dynam_u.append(du_latest.cpu().data.numpy()[0]), item_u.append(all_item.cpu().numpy()),score_u.append(score_up)
        # Logging
        elapsed = time() - start_time; start_time = time()
        print('[Predicting]| Batch {:5d} / {:5d} | seconds/batch {:02.02f}'.format(i, num_batchs, elapsed))
    return id_u, item_u, score_u, dynam_u


def eval_up(uid, pid, dr_model, ub, dr_hidden):
    '''
        calculate latest score for (u,p) pair
        ub for single user
        dr_hidden: should be initialized and repackage after each batch
    '''
    dynamic_user = get_dynamic_u(uid, dr_model, ub, dr_hidden)
    item_embedding = get_item_embedding(pid, dr_model)
    score_up = torch.mm(dynamic_user, item_embedding.t())
    return score_up

def get_dynamic_u(uid, dr_model, ub, dr_hidden):
    '''
        get latest dynamic representation of user uid
        dr_hidden must be provided as global variable
    '''
    for i,x in enumerate(batchify(ub, 1)):
        baskets, lens, uids = x
        _, dynamic_user, _ = dr_model(baskets, lens, dr_hidden)
    return dynamic_user[0][lens[0] - 1].unsqueeze(0)

def get_item_embedding(pid, dr_model):
    '''
        get item's embedding
        pid can be a integer or a torch.cuda.LongTensor
    '''
    if isinstance(pid, torch.cuda.LongTensor) or isinstance(pid, torch.LongTensor):
        return dr_model.encode.weight[pid]
    elif isinstance(pid, int):
        return dr_model.encode.weight[pid].unsqueeze(0)
    else:
        print('Unsupported Index Type %s'%type(pid))
        return None

# Train

In [9]:
# -*- coding: utf-8 -*-
# import constants
# from config import Config
# from dream import DreamModel
# from data import Dataset, BasketConstructor
# from utils import batchify, repackage_hidden, get_grad_norm, get_ratio_update, get_weight_update

import os
import pdb
import torch
import pickle
import random
import numpy as np

from time import time
from math import ceil
from copy import deepcopy
from tensorboardX import SummaryWriter
from sklearn.model_selection import train_test_split

# 測試模型
# from eval import eval_batch

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# BPR損失函數
def bpr_loss(x, dynamic_user, item_embedding, config):
    '''
        bayesian personalized ranking loss for implicit feedback
        parameters:
        - x: batch of users' baskets
        - dynamic_user: batch of users' dynamic representations
        - item_embedding: item_embedding matrix
        - config: model configuration
    '''
    nll = 0
    ub_seqs = []
    for u, du in zip(x, dynamic_user):
        du_p_product = torch.mm(du, item_embedding.t())  # shape: max_len, num_item
        nll_u = []  # nll for user
        for t, basket_t in enumerate(u):
            if basket_t[0] != 0 and t != 0:
                pos_idx = torch.tensor(basket_t, dtype=torch.long).to(device) if config.cuda else torch.tensor(basket_t, dtype=torch.long)
                # Sample negative products
                neg = [random.choice(range(1, config.num_product)) for _ in range(len(basket_t))]  # replacement
                neg_idx = torch.tensor(neg).to(device) if config.cuda else torch.tensor(neg)
                # Score p(u, t, v > v')
                score = du_p_product[t - 1][pos_idx] - du_p_product[t - 1][neg_idx]
                # Average Negative log likelihood for basket_t
                nll_u.append( (- torch.mean(torch.nn.LogSigmoid()(score)) ).unsqueeze(0))
        nll += torch.mean(torch.cat(nll_u))
    return nll

# 訓練模型
def train_dream():
    dr_model.train()  # turn on training mode for dropout
    dr_hidden = dr_model.init_hidden(dr_config.batch_size)
    total_loss = 0
    start_time = time()
    num_batchs = ceil(len(train_ub) / dr_config.batch_size)
    for i, x in enumerate(batchify(train_ub, dr_config.batch_size)):
        baskets, lens, _ = x
        dr_hidden = repackage_hidden(dr_hidden)  # repackage hidden state for RNN
        dr_model.zero_grad()  # optim.zero_grad()
        dynamic_user, _ = dr_model(baskets, lens, dr_hidden)
        loss = bpr_loss(baskets, dynamic_user, dr_model.encode.weight, dr_config)
        loss.backward()

        # Clip to avoid gradient exploding
        torch.nn.utils.clip_grad_norm_(dr_model.parameters(), dr_config.clip)

        # Parameter updating
        # manual SGD
        # for p in dr_model.parameters(): # Update parameters by -lr*grad
        #    p.data.add_(- dr_config.learning_rate, p.grad.data)
        # adam
        grad_norm = get_grad_norm(dr_model)
        previous_params = deepcopy(list(dr_model.parameters()))
        optim.step()

        total_loss += loss.item()
        params = deepcopy(list(dr_model.parameters()))
        delta = get_weight_update(previous_params, params)
        weight_update_ratio = get_ratio_update(delta, params)

        # Logging
        if i % dr_config.log_interval == 0 and i > 0:
            elapsed = (time() - start_time) * 1000 / dr_config.log_interval
            cur_loss = total_loss / dr_config.log_interval / dr_config.batch_size  # turn tensor into float
            total_loss = 0
            start_time = time()
            print(
                '[Training]| Epochs {:3d} | Batch {:5d} / {:5d} | ms/batch {:02.2f} | Loss {:05.2f} |'.format(epoch, i,
                                                                                                              num_batchs,
                                                                                                              elapsed,
                                                                                                              cur_loss))
            writer.add_scalar('model/train_loss', cur_loss, epoch * num_batchs + i)
            writer.add_scalar('model/grad_norm', grad_norm, epoch * num_batchs + i)
            writer.add_scalar('model/weight_update_ratio', weight_update_ratio, epoch * num_batchs + i)

# 驗證模型
def evaluate_dream():
    dr_model.eval()
    dr_hidden = dr_model.init_hidden(dr_config.batch_size)

    total_loss = 0
    start_time = time()
    num_batchs = ceil(len(test_ub) / dr_config.batch_size)
    for i, x in enumerate(batchify(test_ub, dr_config.batch_size)):
        baskets, lens, _ = x
        dynamic_user, _ = dr_model(baskets, lens, dr_hidden)
        loss = bpr_loss(baskets, dynamic_user, dr_model.encode.weight, dr_config)
        dr_hidden = repackage_hidden(dr_hidden)
        total_loss += loss.item()

    # Logging
    elapsed = (time() - start_time) * 1000 / num_batchs
    total_loss = total_loss / num_batchs / dr_config.batch_size
    writer.add_scalar('model/eval_loss', total_loss, (epoch + 1) * num_batchs)
    writer.add_scalar('model/eval_loss', total_loss, (epoch + 1) * num_batchs)
    print('[Evaluation]| Epochs {:3d} | Elapsed {:02.2f} | Loss {:05.2f} |'.format(epoch, elapsed, total_loss))

    return total_loss



# 測試模型
def test_dream(dr_model, data_type, epoch, val_loss):
    
    dr_model.eval()

    # 測試集&其標籤
    with open( FEAT_DATA_DIR + f'./{data_type}/test_ub.pkl', 'rb') as f:
        test_ub = pickle.load(f)
    with open( FEAT_DATA_DIR + f'./{data_type}/test_ub_label.pkl', 'rb') as f:
        test_ub_label = pickle.load(f)

    # print("test_ub_label=",test_ub_label)

    bc = BasketConstructor( RAW_DATA_DIR, FEAT_DATA_DIR )
    # print("ub_basket=\n",test_ub[:5])
    ub = Dataset(test_ub) 
    #up = bc.get_users_products(test_ub, data_type)
    # print("users_products=\n",up)
    res = model_test(dr_model, ub)

    dream_final = res.sort_values(['user_id','dream_score_next'],ascending=[True,False])[['user_id','product_id','dream_score_next']]
    dream_final_list = dream_final.groupby(['user_id'])['product_id'].apply(list)
    dream_final_list.columns = ['user_id', 'product_id']
    k_list = [5,10,30,50,65]
    f1_score_at_k_eval, recall_at_k_eval, precision_at_k_eval = calculate_f1_score_at_k(dream_final_list, k_list)
    ndcg_at_k_eval = calculate_ndcg_at_k(dream_final_list, k_list)
    mae_at_k_eval = calculate_mae_at_k(dream_final_list, k_list)

    f1_5_rec = f1_score_at_k_eval['F1-score@5']
    f1_10_rec = f1_score_at_k_eval['F1-score@10']
    f1_30_rec = f1_score_at_k_eval['F1-score@30']
    f1_50_rec = f1_score_at_k_eval['F1-score@50']
    f1_65_rec = f1_score_at_k_eval['F1-score@65']
    f1_list = [f1_5_rec, f1_10_rec, f1_30_rec, f1_50_rec, f1_65_rec]
    
    recall_5_rec = recall_at_k_eval['Recall@5']
    recall_10_rec = recall_at_k_eval['Recall@10']
    recall_30_rec = recall_at_k_eval['Recall@30']
    recall_50_rec = recall_at_k_eval['Recall@50']
    recall_65_rec = recall_at_k_eval['Recall@65']
    recall_list = [recall_5_rec, recall_10_rec, recall_30_rec, recall_50_rec, recall_65_rec]
    
    precision_5_rec = precision_at_k_eval['Precision@5']
    precision_10_rec = precision_at_k_eval['Precision@10']
    precision_30_rec = precision_at_k_eval['Precision@30']
    precision_50_rec = precision_at_k_eval['Precision@50']
    precision_65_rec = precision_at_k_eval['Precision@65']
    precision_list = [precision_5_rec, precision_10_rec, precision_30_rec, precision_50_rec, precision_65_rec]
    
    ndcg_5_rec = ndcg_at_k_eval['NDCG@5']
    ndcg_10_rec = ndcg_at_k_eval['NDCG@10']
    ndcg_30_rec = ndcg_at_k_eval['NDCG@30']
    ndcg_50_rec = ndcg_at_k_eval['NDCG@50']
    ndcg_65_rec = ndcg_at_k_eval['NDCG@65']
    ndcg_list = [ndcg_5_rec, ndcg_10_rec, ndcg_30_rec, ndcg_50_rec, ndcg_65_rec]
    
    mae_5_rec = mae_at_k_eval['MAE@5']
    mae_10_rec = mae_at_k_eval['MAE@10']
    mae_30_rec = mae_at_k_eval['MAE@30']
    mae_50_rec = mae_at_k_eval['MAE@50']
    mae_65_rec = mae_at_k_eval['MAE@65']
    mae_list = [mae_5_rec, mae_10_rec, mae_30_rec, mae_50_rec, mae_65_rec]
    
    result = [epoch+1] + recall_list + precision_list + f1_list + ndcg_list + mae_list + [val_loss]
    return result


def model_test(dr_model, ub):
    dr_model.config.cuda = True
    id_u, item_u, score_u, dynamic_u = eval_batch(dr_model, ub,  dr_model.config.batch_size , is_reordered = False)
    len_u = [i.shape[0] for i in item_u] # number of products for each user
    flatten_id = np.repeat(id_u, len_u) # repeat id_u to ensure the same length as item_u
    flatten_item = [i for u in item_u for i in u]
    flatten_score = [s for u in score_u for s in u[:-1]]
    res = pd.DataFrame({'user_id': flatten_id, 'product_id': flatten_item, 'dream_score': flatten_score})
    res.drop_duplicates(inplace=True)
    res = res.groupby(['user_id', 'product_id'])['dream_score'].mean().reset_index()
    res.columns = ['user_id', 'product_id', 'dream_score_next']
    return res

def calculate_f1_score_at_k( dream_final_list, k_list ):
    f1_score_at_k_eval = dict()
    recall_at_k_eval = dict()
    precision_at_k_eval = dict()
    for k in k_list:
        f1_score_sum = 0.0
        recall_sum = 0.0
        precision_sum = 0.0
        for i,user_pred_items in enumerate(dream_final_list):
            user_id = dream_final_list.index[i]
            # 預測的TopK結果
            top_k_item_indices = user_pred_items[:k]
            top_k_item_indices = torch.from_numpy(np.array(top_k_item_indices, dtype=np.int64))
            # label_list
            label_list = test_ub_label[test_ub_label['user_id']==user_id]['label'].values[0]
            labels = torch.from_numpy(np.array(label_list, dtype=np.int64))
            # 計算用戶 i 的真實標籤和預測標籤的交集。
            true_positives = torch.sum(torch.sum(torch.eq(top_k_item_indices, labels.unsqueeze(1)).to(torch.float32), dim=1))
            # print("true_positives=",true_positives)
            # 計算用戶 i 的真實標籤和預測標籤的並集。
            predicted_positives = k
            actual_positives = len(labels)
            if actual_positives == 0:
                precision = 0.0
                recall = 0.0
            else:
                precision = true_positives / predicted_positives
                recall = true_positives / actual_positives
            # 計算 F1-score。
            if precision + recall == 0:
                f1_score = 0.0
            else:
                f1_score = 2 * precision * recall / (precision + recall)
            f1_score_sum += f1_score
            recall_sum += recall
            precision_sum += precision
        # 計算平均 F1-score@K 分數。
        f1_score_at_k = float(f1_score_sum) / float(len(dream_final_list))
        recall_at_k = float(recall_sum) / float(len(dream_final_list))
        precision_at_k = float(precision_sum) / float(len(dream_final_list))
        key = '{}@{}'.format('F1-score',k)
        f1_score_at_k_eval[key]=f1_score_at_k
        key = '{}@{}'.format('Recall',k)
        recall_at_k_eval[key]=recall_at_k
        key = '{}@{}'.format('Precision',k)
        precision_at_k_eval[key]=precision_at_k
    return f1_score_at_k_eval, recall_at_k_eval, precision_at_k_eval

def calculate_ndcg_at_k( dream_final_list, k_list ):
    ndcg_at_k_eval = dict()
    for k in k_list:
        ndcg_sum = 0.0
        for i,user_pred_items in enumerate(dream_final_list):
            user_id = dream_final_list.index[i]
            # 預測的TopK結果
            top_k_item_indices = user_pred_items[:k]
            top_k_item_indices = torch.from_numpy(np.array(top_k_item_indices, dtype=np.int64))
            # label_list
            label_list = test_ub_label[test_ub_label['user_id']==user_id]['label'].values[0]
            labels = torch.from_numpy(np.array(label_list, dtype=np.int64))
            
            # 計算 DCG@K。
            if k>len(top_k_item_indices):
                k_ = len(top_k_item_indices)
            else:
                k_ = k

            dcg_at_k = torch.sum(torch.div(1.0, torch.log2(torch.arange(k_, dtype=torch.float32) + 2)) * (torch.eq(top_k_item_indices, labels.unsqueeze(1)).to(torch.float32) / torch.log2(torch.arange(k_, dtype=torch.float32) + 2)))
            # 計算 IDCG@K。
            idcg_at_k = torch.sum(torch.div(1.0, torch.log2(torch.arange(len(labels), dtype=torch.float32) + 2)))
            
            # 計算 NDCG@K。
            ndcg_at_k = dcg_at_k / idcg_at_k
            ndcg_sum += ndcg_at_k.item()

        # 計算平均 NDCG@K 分數。
        ndcg_at_k = ndcg_sum / float(len(dream_final_list))
        key = '{}@{}'.format('NDCG',k)
        ndcg_at_k_eval[key]=ndcg_at_k
    return ndcg_at_k_eval

# MAE
def calculate_mae_at_k(dream_final_list, k_list):
    mae_eval = dict()
    for k in k_list:
        mae_sum = 0.0
        for i,user_pred_items in enumerate(dream_final_list):
            user_id = dream_final_list.index[i]
            # label_list
            label_list = test_ub_label[test_ub_label['user_id']==user_id]['label'].values[0]
            labels = torch.from_numpy(np.array(label_list, dtype=np.int64))
            mae_sum += abs(k - len(labels))
        key = "{}@{}".format("MAE", k)
        mae_eval[key] = mae_sum / float(len(dream_final_list))

    return mae_eval



os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = GPUS

dr_config = Config(DREAM_CONFIG)

rebuild = True # 重建訓練集、測試集


# ---------------------------- #
#       訓練 & 驗證 & 測試      #
# ---------------------------- #

for train_num in range( 1, dr_config.train_times+1 ):   # 跑10次實驗

    # Prepare input
    bc = BasketConstructor(RAW_DATA_DIR, FEAT_DATA_DIR)
    # Users' baskets
    ub_basket = bc.get_baskets( dr_config.data_name, reconstruct=True)
    print("ub_basket=\n",ub_basket)

    # 測試集標籤
    filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/test_ub_label.pkl'
    if  not rebuild and os.path.exists(filepath) :
        with open(filepath, 'rb') as f:
            test_ub_label = pickle.load(f)
        # 測試集
        filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/test_ub.pkl'
        with open(filepath, 'rb') as f:
            test_ub = pickle.load(f)
        # 訓練集
        filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/train_ub.pkl' 
        with open(filepath, 'rb') as f:
            train_ub = pickle.load(f)
    else:
        train_ub, test_ub = train_test_split(ub_basket, test_size=0.2, shuffle=True)
        test_ub = test_ub.reset_index(drop=True)
        #  test_ub : 測試資料，不包含用戶的最後一個購物籃
        #  test_ub_label : 測試資料的標籤(用戶的最後一個購物籃項目ID)
        test_ub_label = test_ub.loc[:, ['user_id','basket']]
        test_ub_label.columns = ['user_id','label']

        for i,bsk in enumerate(test_ub["basket"]):
            test_ub_label["label"][i] = test_ub[ "basket"][i][-1]
            test_ub["basket"][i] = test_ub["basket"][i][:-1]
        
        filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/test_ub_label.pkl'
        # 測試集標籤
        with open(filepath, 'wb') as f:
            pickle.dump(test_ub_label, f, pickle.HIGHEST_PROTOCOL)
        # 測試集
        filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/test_ub.pkl' 
        with open(filepath, 'wb') as f:
            pickle.dump(test_ub, f, pickle.HIGHEST_PROTOCOL)
        # 訓練集
        filepath = FEAT_DATA_DIR + f'/{dr_config.data_name}/train_ub.pkl' 
        with open(filepath, 'wb') as f:
            pickle.dump(train_ub, f, pickle.HIGHEST_PROTOCOL)
    
    print("test_ub_label=",test_ub_label)
    print("ub_basket=\n",test_ub[:5])
        
    del ub_basket

    dir_path = os.getcwd() + DREAM_MODEL_DIR + f'/{dr_config.data_name}'
    train_ub, test_ub = Dataset(train_ub), Dataset(test_ub)

    # Model config
    dr_model = DreamModel(dr_config)
    if dr_config.cuda:
        dr_model.cuda()

    # Optimizer
    optim = torch.optim.Adam(dr_model.parameters(), lr = dr_config.learning_rate)
    # optim = torch.optim.Adadelta(dr_model.parameters())
    # optim = torch.optim.SGD(dr_model.parameters(), lr=dr_config.learning_rate, momentum=0.9)
    writer = SummaryWriter(log_dir= 'runs/{}/{}'.format(dr_config.data_name, dr_config.alias))  # tensorboard writer
    writer.add_text('config', str(dr_config))
    best_val_loss = None

    try:
        for k,v in DREAM_CONFIG.items():
            print(k,v)
        # training
        results = []
        for epoch in range(dr_config.epochs):
            train_dream()
            print('-' * 89)
            val_loss = evaluate_dream()
            print('-' * 89)
            # checkpoint
            path = DREAM_MODEL_DIR + f'/{dr_config.data_name}/'
            if not os.path.isdir(path):
                os.mkdir(path)
            # with open(dr_config.checkpoint_dir.format( df = dr_config.df_num, train_num = train_num ,epoch = epoch, loss = val_loss), 'wb') as f:
            #     torch.save(dr_model, f)
            
            # 測試模型
            result = test_dream(dr_model, dr_config.data_name , epoch, val_loss)
            results.append(result)
            print("result=", result)
            record_df = pd.DataFrame(results,columns=['Epoch','Recall@5', 'Recall@10', 'Recall@30', 'Recall@50', 'Recall@65',
                                                      'Precision@5', 'Precision@10', 'Precision@30', 'Precision@50', 'Precision@65',
                                                      'F1-score@5', 'F1-score@10', 'F1-score@30', 'F1-score@50', 'F1-score@65',
                                                      'NDCG@5', 'NDCG@10', 'NDCG@30', 'NDCG@50', 'NDCG@65',
                                                      'MAE@5', 'MAE@10', 'MAE@30', 'MAE@50', 'MAE@65', 'loss'])
            path = dir_path + f'/output_{train_num}.csv'
            record_df.to_csv(path, index=False)

    except KeyboardInterrupt:
        print('*' * 89)
        print('Early Stopping!')
        break



# 執行時，當前目錄須移至"\程式碼\ASBRec_Code\baseline\DREAM"
# python src\train.py

filepath= ./tmp/feat//Dunnhumby/basket_Dunnhumby.pkl
ub_basket=
        user_id                                             basket
0           31  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [11, 12, ...
1           68  [[51, 52, 53, 54], [55, 56, 57, 58], [56, 59, ...
2          180  [[11, 62, 122, 151, 152, 153, 154, 155, 156, 1...
3          324  [[0, 41, 110, 139, 231, 242, 243, 244, 245, 24...
4          358  [[0, 11, 31, 63, 158, 172, 302, 307, 308, 309,...
...        ...                                                ...
12821   999696  [[130, 172], [130, 1668, 1925, 2679], [19, 130...
12822   999698  [[162, 708, 778, 1123, 1502, 1516, 1829], [56,...
12823   999718  [[16, 107, 138, 158, 274, 280, 290, 558, 568, ...
12824   999934  [[0, 11, 62, 67, 108, 109, 140, 240, 245, 379,...
12825   999976  [[11, 62, 903, 1172, 1977, 2228, 2934], [290, ...

[12826 rows x 2 columns]
test_ub_label=       user_id                                              label
0      584429                        

[Training]| Epochs   0 | Batch   150 /   321 | ms/batch 9437.37 | Loss 00.68 |
[Training]| Epochs   0 | Batch   152 /   321 | ms/batch 10175.41 | Loss 00.68 |
[Training]| Epochs   0 | Batch   154 /   321 | ms/batch 6266.12 | Loss 00.68 |
[Training]| Epochs   0 | Batch   156 /   321 | ms/batch 11920.95 | Loss 00.68 |
[Training]| Epochs   0 | Batch   158 /   321 | ms/batch 11724.85 | Loss 00.68 |
[Training]| Epochs   0 | Batch   160 /   321 | ms/batch 10747.18 | Loss 00.68 |
[Training]| Epochs   0 | Batch   162 /   321 | ms/batch 8254.67 | Loss 00.68 |
[Training]| Epochs   0 | Batch   164 /   321 | ms/batch 8342.36 | Loss 00.68 |
[Training]| Epochs   0 | Batch   166 /   321 | ms/batch 10326.36 | Loss 00.68 |
[Training]| Epochs   0 | Batch   168 /   321 | ms/batch 14446.86 | Loss 00.69 |
[Training]| Epochs   0 | Batch   170 /   321 | ms/batch 8519.05 | Loss 00.69 |
[Training]| Epochs   0 | Batch   172 /   321 | ms/batch 8951.11 | Loss 00.69 |
[Training]| Epochs   0 | Batch   174 /   321 |

[Predicting]| Batch    21 /    81 | seconds/batch 1.43
[Predicting]| Batch    22 /    81 | seconds/batch 0.82
[Predicting]| Batch    23 /    81 | seconds/batch 1.13
[Predicting]| Batch    24 /    81 | seconds/batch 0.68
[Predicting]| Batch    25 /    81 | seconds/batch 1.28
[Predicting]| Batch    26 /    81 | seconds/batch 2.03
[Predicting]| Batch    27 /    81 | seconds/batch 1.17
[Predicting]| Batch    28 /    81 | seconds/batch 1.10
[Predicting]| Batch    29 /    81 | seconds/batch 1.12
[Predicting]| Batch    30 /    81 | seconds/batch 0.78
[Predicting]| Batch    31 /    81 | seconds/batch 0.98
[Predicting]| Batch    32 /    81 | seconds/batch 0.58
[Predicting]| Batch    33 /    81 | seconds/batch 1.20
[Predicting]| Batch    34 /    81 | seconds/batch 1.33
[Predicting]| Batch    35 /    81 | seconds/batch 1.43
[Predicting]| Batch    36 /    81 | seconds/batch 1.26
[Predicting]| Batch    37 /    81 | seconds/batch 1.14
[Predicting]| Batch    38 /    81 | seconds/batch 1.12
[Predictin

[Training]| Epochs   1 | Batch   112 /   321 | ms/batch 3939.73 | Loss 00.62 |
[Training]| Epochs   1 | Batch   114 /   321 | ms/batch 5094.70 | Loss 00.64 |
[Training]| Epochs   1 | Batch   116 /   321 | ms/batch 7065.99 | Loss 00.64 |
[Training]| Epochs   1 | Batch   118 /   321 | ms/batch 4726.23 | Loss 00.66 |
[Training]| Epochs   1 | Batch   120 /   321 | ms/batch 5311.50 | Loss 00.66 |
[Training]| Epochs   1 | Batch   122 /   321 | ms/batch 6216.62 | Loss 00.64 |
[Training]| Epochs   1 | Batch   124 /   321 | ms/batch 5066.72 | Loss 00.65 |
[Training]| Epochs   1 | Batch   126 /   321 | ms/batch 4883.07 | Loss 00.65 |
[Training]| Epochs   1 | Batch   128 /   321 | ms/batch 6202.71 | Loss 00.65 |
[Training]| Epochs   1 | Batch   130 /   321 | ms/batch 6002.09 | Loss 00.65 |
[Training]| Epochs   1 | Batch   132 /   321 | ms/batch 4627.66 | Loss 00.63 |
[Training]| Epochs   1 | Batch   134 /   321 | ms/batch 5544.89 | Loss 00.65 |
[Training]| Epochs   1 | Batch   136 /   321 | ms/ba

32
[Training]| Epochs   1 | Batch   320 /   321 | ms/batch 5796.86 | Loss 00.63 |
-----------------------------------------------------------------------------------------
32
[Evaluation]| Epochs   1 | Elapsed 1861.14 | Loss 00.62 |
-----------------------------------------------------------------------------------------
[Predicting]| Batch     0 /    81 | seconds/batch 0.62
[Predicting]| Batch     1 /    81 | seconds/batch 0.75
[Predicting]| Batch     2 /    81 | seconds/batch 0.91
[Predicting]| Batch     3 /    81 | seconds/batch 0.79
[Predicting]| Batch     4 /    81 | seconds/batch 0.67
[Predicting]| Batch     5 /    81 | seconds/batch 0.53
[Predicting]| Batch     6 /    81 | seconds/batch 0.73
[Predicting]| Batch     7 /    81 | seconds/batch 0.47
[Predicting]| Batch     8 /    81 | seconds/batch 0.61
[Predicting]| Batch     9 /    81 | seconds/batch 0.53
[Predicting]| Batch    10 /    81 | seconds/batch 0.36
[Predicting]| Batch    11 /    81 | seconds/batch 0.34
[Predicting]| Bat

...

[Training]| Epochs  37 | Batch   310 /   321 | ms/batch 4080.07 | Loss 00.56 |
[Training]| Epochs  37 | Batch   312 /   321 | ms/batch 5752.25 | Loss 00.59 |
[Training]| Epochs  37 | Batch   314 /   321 | ms/batch 5844.44 | Loss 00.59 |
[Training]| Epochs  37 | Batch   316 /   321 | ms/batch 5699.62 | Loss 00.59 |
[Training]| Epochs  37 | Batch   318 /   321 | ms/batch 4865.47 | Loss 00.58 |
32
[Training]| Epochs  37 | Batch   320 /   321 | ms/batch 5218.69 | Loss 00.58 |
-----------------------------------------------------------------------------------------
32
[Evaluation]| Epochs  37 | Elapsed 1443.05 | Loss 00.58 |
-----------------------------------------------------------------------------------------
[Predicting]| Batch     0 /    81 | seconds/batch 0.39
[Predicting]| Batch     1 /    81 | seconds/batch 0.55
[Predicting]| Batch     2 /    81 | seconds/batch 0.36
[Predicting]| Batch     3 /    81 | seconds/batch 0.75
[Predicting]| Batch     4 /    81 | seconds/batch 0.42
[Predic

[Training]| Epochs  38 | Batch    66 /   321 | ms/batch 4350.87 | Loss 00.56 |
[Training]| Epochs  38 | Batch    68 /   321 | ms/batch 4833.63 | Loss 00.58 |
[Training]| Epochs  38 | Batch    70 /   321 | ms/batch 4577.68 | Loss 00.57 |
[Training]| Epochs  38 | Batch    72 /   321 | ms/batch 3638.92 | Loss 00.56 |
[Training]| Epochs  38 | Batch    74 /   321 | ms/batch 4492.97 | Loss 00.57 |
[Training]| Epochs  38 | Batch    76 /   321 | ms/batch 4766.02 | Loss 00.56 |
[Training]| Epochs  38 | Batch    78 /   321 | ms/batch 5594.92 | Loss 00.58 |
[Training]| Epochs  38 | Batch    80 /   321 | ms/batch 5039.83 | Loss 00.57 |
[Training]| Epochs  38 | Batch    82 /   321 | ms/batch 5573.65 | Loss 00.59 |
[Training]| Epochs  38 | Batch    84 /   321 | ms/batch 4096.04 | Loss 00.55 |
[Training]| Epochs  38 | Batch    86 /   321 | ms/batch 5800.39 | Loss 00.60 |
[Training]| Epochs  38 | Batch    88 /   321 | ms/batch 5871.32 | Loss 00.58 |
[Training]| Epochs  38 | Batch    90 /   321 | ms/ba

[Training]| Epochs  38 | Batch   274 /   321 | ms/batch 5731.17 | Loss 00.59 |
[Training]| Epochs  38 | Batch   276 /   321 | ms/batch 4463.85 | Loss 00.56 |
[Training]| Epochs  38 | Batch   278 /   321 | ms/batch 5906.90 | Loss 00.59 |
[Training]| Epochs  38 | Batch   280 /   321 | ms/batch 4853.65 | Loss 00.58 |
[Training]| Epochs  38 | Batch   282 /   321 | ms/batch 4788.95 | Loss 00.57 |
[Training]| Epochs  38 | Batch   284 /   321 | ms/batch 4388.77 | Loss 00.55 |
[Training]| Epochs  38 | Batch   286 /   321 | ms/batch 5186.89 | Loss 00.58 |
[Training]| Epochs  38 | Batch   288 /   321 | ms/batch 5782.58 | Loss 00.59 |
[Training]| Epochs  38 | Batch   290 /   321 | ms/batch 5252.18 | Loss 00.57 |
[Training]| Epochs  38 | Batch   292 /   321 | ms/batch 4259.22 | Loss 00.57 |
[Training]| Epochs  38 | Batch   294 /   321 | ms/batch 4910.15 | Loss 00.58 |
[Training]| Epochs  38 | Batch   296 /   321 | ms/batch 4100.63 | Loss 00.56 |
[Training]| Epochs  38 | Batch   298 /   321 | ms/ba

[Training]| Epochs  39 | Batch    30 /   321 | ms/batch 4569.23 | Loss 00.56 |
[Training]| Epochs  39 | Batch    32 /   321 | ms/batch 4350.47 | Loss 00.56 |
[Training]| Epochs  39 | Batch    34 /   321 | ms/batch 4451.05 | Loss 00.55 |
[Training]| Epochs  39 | Batch    36 /   321 | ms/batch 5566.35 | Loss 00.58 |
[Training]| Epochs  39 | Batch    38 /   321 | ms/batch 5649.18 | Loss 00.58 |
[Training]| Epochs  39 | Batch    40 /   321 | ms/batch 5285.54 | Loss 00.58 |
[Training]| Epochs  39 | Batch    42 /   321 | ms/batch 5079.47 | Loss 00.59 |
[Training]| Epochs  39 | Batch    44 /   321 | ms/batch 4988.71 | Loss 00.58 |
[Training]| Epochs  39 | Batch    46 /   321 | ms/batch 4372.72 | Loss 00.56 |
[Training]| Epochs  39 | Batch    48 /   321 | ms/batch 5411.99 | Loss 00.58 |
[Training]| Epochs  39 | Batch    50 /   321 | ms/batch 4493.23 | Loss 00.57 |
[Training]| Epochs  39 | Batch    52 /   321 | ms/batch 4740.18 | Loss 00.57 |
[Training]| Epochs  39 | Batch    54 /   321 | ms/ba

[Training]| Epochs  39 | Batch   238 /   321 | ms/batch 4705.48 | Loss 00.55 |
[Training]| Epochs  39 | Batch   240 /   321 | ms/batch 3848.12 | Loss 00.56 |
[Training]| Epochs  39 | Batch   242 /   321 | ms/batch 3521.17 | Loss 00.51 |
[Training]| Epochs  39 | Batch   244 /   321 | ms/batch 5448.51 | Loss 00.60 |
[Training]| Epochs  39 | Batch   246 /   321 | ms/batch 4736.73 | Loss 00.56 |
[Training]| Epochs  39 | Batch   248 /   321 | ms/batch 5510.90 | Loss 00.58 |
[Training]| Epochs  39 | Batch   250 /   321 | ms/batch 5096.93 | Loss 00.58 |
[Training]| Epochs  39 | Batch   252 /   321 | ms/batch 4789.09 | Loss 00.58 |
[Training]| Epochs  39 | Batch   254 /   321 | ms/batch 4953.67 | Loss 00.56 |
[Training]| Epochs  39 | Batch   256 /   321 | ms/batch 5402.64 | Loss 00.58 |
[Training]| Epochs  39 | Batch   258 /   321 | ms/batch 3816.76 | Loss 00.55 |
[Training]| Epochs  39 | Batch   260 /   321 | ms/batch 4487.68 | Loss 00.56 |
[Training]| Epochs  39 | Batch   262 /   321 | ms/ba